<a href="https://colab.research.google.com/github/emanbuc/ASL-Recognition-Deep-Learning/blob/main/FastAI_Static_Gesture_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai --upgrade -q


     |████████████████████████████████| 204kB 7.8MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 


In [30]:
from fastai.vision.all import *
from google.colab import drive
from google.colab import drive
drive.mount('/gdrive',force_remount=true)


Mounted at /gdrive


In [45]:
root_dir = '/content/drive/MyDrive/Gesture/'
imagePath=Path(root_dir + 'images')

In [46]:
imagePath

Path('/content/drive/MyDrive/Gesture/images')

In [48]:
import os



files = os.listdir(imagePath)

for f in files:
	print(f)


test_image_FS_A_221.png
test_image_FS_A_224.png
test_image_FS_A_223.png
test_image_FS_A_222.png
test_image_FS_A_225.png
test_image_FS_A_230.png
test_image_FS_A_228.png
test_image_FS_A_229.png
test_image_FS_A_226.png
test_image_FS_A_227.png
test_image_FS_A_231.png
test_image_FS_B_231.png
test_image_FS_B_234.png
test_image_FS_B_232.png
test_image_FS_B_233.png
test_image_FS_B_239.png
test_image_FS_B_235.png
test_image_FS_B_237.png
test_image_FS_B_238.png
test_image_FS_B_236.png
test_image_FS_B_242.png
test_image_FS_B_244.png
test_image_FS_B_241.png
test_image_FS_B_245.png
test_image_FS_B_240.png
test_image_FS_B_243.png
test_image_FS_C_391.png
test_image_FS_C_393.png
test_image_FS_C_394.png
test_image_FS_C_395.png
test_image_FS_C_392.png
test_image_FS_C_396.png
test_image_FS_C_397.png
test_image_FS_C_398.png
test_image_FS_C_399.png
test_image_FS_C_400.png
test_image_FS_C_401.png


In [49]:
fnames = get_image_files(imagePath)
fnames

(#37) [Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_221.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_224.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_223.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_222.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_225.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_230.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_228.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_229.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_226.png'),Path('/content/drive/MyDrive/Gesture/images/test_image_FS_A_227.png')...]

In [50]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=Pipeline([attrgetter("name"),RegexLabeller(pat = r'^.*_([A-Z])_\d+.png$')]),
                   splitter=RandomSplitter(valid_pct=0.2, seed=42),
                   item_tfms=RandomResizedCrop(224, min_scale=0.5),
                   batch_tfms=aug_transforms())

In [53]:
dsets = dblock.datasets(imagePath)


(PILImage mode=RGB size=375x277, TensorCategory(1))

In [54]:
dsets.train[0]


(PILImage mode=RGB size=375x277, TensorCategory(1))

In [ ]:
dsets.valid[0]

In [55]:
dsets

AttributeError: ignored

# Nuova sezione